https://xywang93.github.io/2018/03/02/DeepLearning/TensorFlow/02Tensorflow%E5%9F%BA%E6%9C%AC%E6%A8%A1%E5%9E%8B%E4%B9%8B%E7%BA%BF%E6%80%A7%E5%9B%9E%E5%BD%92/
Tensorflow基本模型之线性回归
 发表于 2018-03-02 |  分类于 Deep Learning ， TensorFlow |  | 本文总阅读量 次
 字数统计: 1,246

In [6]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
rng = np.random


In [4]:
# 参数设置
learning_rate = 0.01
training_epochs = 1000
display_step = 50
# 生成训练数据
train_X = np.asarray([3.3,4.4,5.5,6.71,6.93,4.168,9.779,6.182,7.59,2.167,
                         7.042,10.791,5.313,7.997,5.654,9.27,3.1])
train_Y = np.asarray([1.7,2.76,2.09,3.19,1.694,1.573,3.366,2.596,2.53,1.221,
                         2.827,3.465,1.65,2.904,2.42,2.94,1.3])
n_samples = train_X.shape[0]
# tf 图的输入
X = tf.placeholder('float')
Y = tf.placeholder('float')
# 设置模型的权重与偏置
W = tf.Variable(rng.randn(), name='weight')
b = tf.Variable(rng.randn(), name='bias')

#构造一个线性模型
# pred = tf.add(tf.multiply(X, W), b) #仅在下面 cost 引用一次
pred = X * W + b #可以, 上面代码为什么要这么繁琐?

#损失函数设置为均方误差
cost = tf.reduce_sum(tf.pow(pred - Y, 2)) / (2 * n_samples)

'定义优化方法'
#梯度下降
optimizer = tf.train.GradientDescentOptimizer(learning_rate)
train_op = optimizer.minimize(cost)

#开始训练
with tf.Session() as sess:
    init = tf.global_variables_initializer()
    sess.run(init)
    
    # Fit all training data
    for epoch in range(training_epochs):
        for (x, y) in zip(train_X, train_Y):
            sess.run(train_op, feed_dict = {X: x, Y: y})
            
        if (epoch + 1) % display_step == 0:
            c = sess.run(cost, feed_dict = {X: train_X, Y: train_Y})
            print('epoch:', '%04d' % (epoch+1), 'cost:', '%.9f' % c, \
                 'W=', sess.run(W), 'b=', sess.run(b))
            
    print('opt finished---------------')
    training_cost = sess.run(cost, feed_dict = {X: train_X, Y: train_Y})
    print('training_cost', training_cost, 'W=', sess.run(W), 'b=', sess.run(b))
    
    #plot
    plt.plot(train_X, train_Y, 'ro', label='Original data')
    plt.plot(train_X, sess.run(W) * train_X + sess.run(b), label='Fitted line')
    plt.legend()
    #plt.show()

epoch: 0050 cost: 0.148054853 W= 0.39869916 b= -0.27118853
epoch: 0100 cost: 0.139840752 W= 0.3898312 b= -0.20739265
epoch: 0150 cost: 0.132575378 W= 0.38149053 b= -0.14739087
epoch: 0200 cost: 0.126149252 W= 0.37364605 b= -0.09095788


KeyboardInterrupt: 

下面代码是使用 eager API，不用 session run
--

from： E:\Github\TensorFlow-Examples\examples\2_BasicModels\linear_regression_eager_api.py

In [1]:
'''
需要重启 kernel, 否则报错:
ValueError: tf.enable_eager_execution must be called at program startup.
'''
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf

# Set Eager API
tf.enable_eager_execution()
tfe = tf.contrib.eager


In [2]:
'参数、数据与上面 cell 保持一致'
# 参数设置
learning_rate = 0.01
training_epochs = 1000
display_step = 50
# 生成训练数据
train_X = np.asarray([3.3,4.4,5.5,6.71,6.93,4.168,9.779,6.182,7.59,2.167,
                         7.042,10.791,5.313,7.997,5.654,9.27,3.1])
train_Y = np.asarray([1.7,2.76,2.09,3.19,1.694,1.573,3.366,2.596,2.53,1.221,
                         2.827,3.465,1.65,2.904,2.42,2.94,1.3])
n_samples = train_X.shape[0]

In [28]:
#W, b
W = tfe.Variable(rng.randn(), name='weight')
b = tfe.Variable(rng.randn(), name='bias')

#构造一个线性模型
# pred = tf.add(tf.multiply(X, W), b) #仅在下面 cost 引用一次
# pred = X * W + b #可以, 上面代码为什么要这么繁琐?
def pred(inputs):
    return inputs * W + b

#损失函数设置为均方误差
# cost = tf.reduce_sum(tf.pow(pred - Y, 2)) / (2 * n_samples)
cost = lambda func, X, Y: tf.reduce_sum(tf.pow(func(X) - Y, 2)) / (2 * n_samples)

'定义优化方法'
#梯度下降, 说是 SGD
optimizer = tf.train.GradientDescentOptimizer(learning_rate)
# train_op = optimizer.minimize(cost)

grad = tfe.implicit_gradients(cost)

print('init cost= %f' % cost(pred, train_X, train_Y), 
      '; W, b=', W.numpy(), b.numpy() )

for epoch in range(training_epochs):
    '#前面说 SGD, 这里怎么用全样本?'
    optimizer.apply_gradients(grad(pred, train_X, train_Y))
    
    if (epoch + 1) % display_step == 0 or epoch == 0:
        print('epoch: %04d, cost= %.9f, W, b= %f, %f' % 
              (epoch + 1, cost(pred, train_X, train_Y), W.numpy(), b.numpy()) )

# Graphic display
plt.plot(train_X, train_Y, 'ro', label='Original data')
# plt.plot(train_X, np.array(W * train_X + b), label='Fitted line')
plt.plot(train_X, W * train_X + b, label='Fitted line') #不转 np-arr 也行！
plt.legend()
plt.show()


init cost= 9.994187 ; W, b= 0.94785094 0.6066371
epoch: 0001, cost= 3.062878847, W, b= 0.651521, 0.565191
epoch: 0050, cost= 0.081372276, W, b= 0.289427, 0.530873
epoch: 0100, cost= 0.080864072, W, b= 0.287200, 0.546660
epoch: 0150, cost= 0.080414012, W, b= 0.285105, 0.561516
epoch: 0200, cost= 0.080015406, W, b= 0.283133, 0.575497
epoch: 0250, cost= 0.079662398, W, b= 0.281277, 0.588655
epoch: 0300, cost= 0.079349756, W, b= 0.279530, 0.601037
epoch: 0350, cost= 0.079072878, W, b= 0.277887, 0.612689
epoch: 0400, cost= 0.078827649, W, b= 0.276340, 0.623655
epoch: 0450, cost= 0.078610510, W, b= 0.274884, 0.633975
epoch: 0500, cost= 0.078418165, W, b= 0.273514, 0.643687
epoch: 0550, cost= 0.078247853, W, b= 0.272225, 0.652826
epoch: 0600, cost= 0.078096978, W, b= 0.271012, 0.661427
epoch: 0650, cost= 0.077963382, W, b= 0.269870, 0.669521
epoch: 0700, cost= 0.077845067, W, b= 0.268796, 0.677139
epoch: 0750, cost= 0.077740259, W, b= 0.267785, 0.684307
epoch: 0800, cost= 0.077647470, W, b= 0

In [3]:
tfe.implicit_gradients?


In [4]:
tfe.*grad*?
'''
共有：
tfe.custom_gradient
tfe.gradients_function
tfe.implicit_gradients
tfe.implicit_value_and_gradients
tfe.value_and_gradients_function
'''

In [32]:
tfe.custom_gradient?